# The Anatomy of a Python Class (Part II)

> Author: Samuel Farrens (<samuel.farrens@cea.fr>)  
> Year: 2019

Classes are one of the fundamental building blocks of Python and are essential for object-oriented programming.

In this notebook we will explore how classes work and look at tips and tricks for getting the most out of them. By the end of this tutorial you should have, not only a much better understanding of Python classes, but also some new ideas for writing better code.

## Contents

---

* [1 Inheritance](#1-Inheritance)
  * [Inherting Attributes](#Inherting-Attributes)
  * [Overriding](#Overriding)
  * [Handling Instantiation](#Handling-Instantiation)
  * [Multiple Parents](#Multiple-Parents)
  * [Method Resolution Order](#Method-Resolution-Order)
* [2 Composition](#2-Composition)
* [3 Abstract Classes](#3-Abstract-Classes)
  * [Abstract Methods](#Abstract-Methods)
  * [Abstract Properties](#Abstract-Properties)


## 1 Inheritance

---

<br>

A powerful property of classes is the ability to inherit attributes and methods from other classes. This helps avoid repetition of code, which makes it easier to develop and maintain. 

<br>

### Inherting Attributes

We will begin by defining a simple class with a single static method that will act as a *parent* class.

In [1]:
# Define a parent class
class Parent:
    
    @staticmethod
    def add(a, b):
        return a + b

# Print the parent dictionary
print('Parent.__dict__ =', Parent.__dict__)

Parent.__dict__ = {'__module__': '__main__', 'add': <staticmethod object at 0x111676fd0>, '__dict__': <attribute '__dict__' of 'Parent' objects>, '__weakref__': <attribute '__weakref__' of 'Parent' objects>, '__doc__': None}


Note that this class has no special properties, it the same as classes we saw in the previous notebook.

Now we will define a second class, also with a single static function. This class will serve as a *child* class that will inherit from the parent class. This is done by simply passing the parent class name in `()` when defining the child class. 

In [2]:
# Define a child class
class Child(Parent):
    
    @staticmethod
    def subtract(a, b):
        return a - b

print('Child.__dict__ =', Child.__dict__)

Child.__dict__ = {'__module__': '__main__', 'subtract': <staticmethod object at 0x11169c470>, '__doc__': None}


We can see that the contents of the child class dictionary are only what is expected from a normal class definition. Using the special `__bases__` attribute, however, we can see a tuple of classes that the child inherits from.

In [3]:
print('Child.__bases__ =', Child.__bases__)

Child.__bases__ = (<class '__main__.Parent'>,)


Finally, we can demonstrate that the child has indeed inherited new attributes from the parent as follows.

In [4]:
print('1 + 2 =', Child.add(1, 2))
print('3 - 2 =', Child.subtract(3, 2))

1 + 2 = 3
3 - 2 = 1


We can clearly see that `Child` has inhertited the static `add` method from `Parent`.

In the following example, we can see that this works for any type of class attribute.

In [5]:
class Parent:
    
    x = 1
    
class Child(Parent):
    
    @classmethod
    def show(cls):
        return print('x =', cls.x)

Child.show()

x = 1


### Overriding

We saw in the previous notebook that class intance attributes will override class attributes of the same name. The same happens with child attributes with respect to parent attributes. 

In [6]:
class Parent:
    
    x = 1
    
class Child(Parent):
    
    x = 2
    
    @classmethod
    def show(cls):
        return print('x =', cls.x)

Child.show()

x = 2


### Hierachy

It is possible to define a hierchy of parent classes, the attributes of which will be inheried by a given child class.

In [8]:
class GrandParent:
    
    x = 1
    name = 'Grandparent'

class Parent(GrandParent):
    
    y = 2
    name = 'Parent'
    
class Child(Parent):
    
    z = 3
    name = 'Child'

print(Child.name, Child.x, Child.y, Child.z)

Child 1 2 3


Note that overriding will also act hierachically, meaning that the last class in the chain will be given precedence.

<br>

### Handling Instantiation 

If the parent class contains an `__init__` method, this too can be inherited by the child.

In [9]:
class Parent:
    
    def __init__(self, value):
        self.myattr = value
    
class Child(Parent):
    
    def show(self):
        return print('myattr =', self.myattr)
    
Child('A string').show()

myattr = A string


In fact it also works the other way around.

In [10]:
class Parent:
    
    def show(self):
        return print('myattr =', self.myattr)
    
class Child(Parent):
    
    def __init__(self, value):
        self.myattr = value
    
Child('A string').show()

myattr = A string


This is because the child will lookup any attributes not listed in its own dictionary in all the parent (or base) classes.

If both classes have an `__init__` method only the child class will be initialised (see [Overriding](#Overriding)).

In [11]:
class Parent:
    
    def __init__(self):
        self.pval = 'parent value'
    
class Child(Parent):
    
    def __init__(self):
        self.cval = 'child value'
    
inst = Child()
print('cval =', inst.cval)
print('pval =', inst.pval)

cval = child value


AttributeError: 'Child' object has no attribute 'pval'

We can fix this by manually instantiating the parent class.

In [12]:
class Parent:
    
    def __init__(self):
        self.pval = 'parent value'
    
class Child(Parent):
    
    def __init__(self):
        self.cval = 'child value'
        Parent.__init__(self)
    
inst = Child()
print('cval =', inst.cval)
print('pval =', inst.pval)

cval = child value
pval = parent value


Python has a useful shortcut to make this process easier using the [`super`](https://docs.python.org/3/library/functions.html?highlight=super#super)function. This allows us to instantiate the parent without naming it, this particularly useful if the name of the parent class were to change.

In [13]:
class Parent:
    
    def __init__(self):
        self.pval = 'parent value'
    
class Child(Parent):
    
    def __init__(self):
        self.cval = 'child value'
        super().__init__()
    
inst = Child()
print('cval =', inst.cval)
print('pval =', inst.pval)

cval = child value
pval = parent value


### Multiple Parents

It is possible for a child to inherit attributes from multiple parents.

In [14]:
class Mother:
    
    def __init__(self, value):
        self.mval = value
        
    def show_mother(self):
        print('mother value =', self.mval)
        
class Father:
    
    def __init__(self, value):
        self.fval = value
        
    def show_father(self):
        print('father value =', self.fval)
    
class Child(Mother, Father):
    
    def __init__(self, value1, value2, value3):
        self.cval = value3
        Mother.__init__(self, value1)
        Father.__init__(self, value2)
        
    def show_child(self):
        print('child value =', self.cval)
        
inst = Child(1, 2, 3)
inst.show_mother()
inst.show_father()
inst.show_child()

mother value = 1
father value = 2
child value = 3


Note that the parent classes were explicitly instantiated in the previous example. Attempting the same thing using the `super` function will raise an error.

In [15]:
class Mother:
    
    def __init__(self, value):
        self.mval = value
        
    def show_mother(self):
        print('mother value =', self.mval)
        
class Father:
    
    def __init__(self, value):
        self.fval = value
        
    def show_father(self):
        print('father value =', self.fval)
    
class Child(Mother, Father):
    
    def __init__(self, value1, value2):
        self.cval = value2
        super().__init__(value1)
        
    def show_child(self):
        print('child value =', self.cval)
        
inst = Child(1, 2)
inst.show_mother()
inst.show_father()
inst.show_child()

mother value = 1


AttributeError: 'Child' object has no attribute 'fval'

As we can see the `Mother` parent class was properly instantiated, but the `Father` parent class was not. The reasons for this are explained in the following subsection.

> In general, caution should be used when building a class archtecture that requires inheritance for multiple parent classes.

<br>

### Method Resolution Order

The Method Resolution Order (MRO) dictates the order in which the child class will search the base classes for a given attribute. We can see the MRO of the child class from the previous example using the `__mro__` attribute as follows.

In [16]:
print('Child.__mro__ =', Child.__mro__)

Child.__mro__ = (<class '__main__.Child'>, <class '__main__.Mother'>, <class '__main__.Father'>, <class 'object'>)


We can see that the `Mother` class appears before the `Father` class in the MRO, hence the `super` method in `Child` instantiates `Mother`. 

In order to make the previous example work, we would need to add another `super` in the `__init__` of the `Mother` class as follows.

> Note: Don't do this in your code!

In [17]:
class Mother:
    
    def __init__(self, value):
        self.mval = value
        super().__init__(value)
        
    def show_mother(self):
        print('mother value =', self.mval)
        
class Father:
    
    def __init__(self, value):
        self.fval = value
        
    def show_father(self):
        print('father value =', self.fval)
    
class Child(Mother, Father):
    
    def __init__(self, value1, value2):
        self.cval = value2
        super().__init__(value1)
        
    def show_child(self):
        print('child value =', self.cval)
        
inst = Child(1, 2)
inst.show_mother()
inst.show_father()
inst.show_child()

mother value = 1
father value = 1
child value = 2


This, however, is not recommended as it adds unncessary abiguity to the code and makes debugging extremely difficult.

<br>

## 2 Composition
---

<br>

Inhertitance is not the only way in which a class can access attributes from other classes. This comes back one of main take-away messages of this tutorial, namely that everything in Python is an object and any object can be assigned as an attribute.

This means that a class attribute can actually be an instance of another class.

In [18]:
class Composer:
    
    def __init__(self):
        self.myattr = 'composer value'
    
class myClass:
    
    def __init__(self):
        self.comp = Composer()

inst = myClass()
print("myattr =", inst.comp.myattr)

myattr = composer value


This can particularly useful as we can pass classes or class intances to our new class without knowing anything about the other classes. 

In [19]:
class Star:
    
    def __init__(self):
        self.whoami = 'I am a star!'
        
class Galaxy:
    
    def __init__(self):
        self.whoami = 'I am a Galaxy!'
    
class myClass:
    
    def __init__(self, composer):
        self.comp = composer()

inst1 = myClass(Star)
inst2 = myClass(Galaxy)
print("whoami =", inst1.comp.whoami)
print("whoami =", inst2.comp.whoami)

whoami = I am a star!
whoami = I am a Galaxy!


We can even achieve inheritance-like properties if we know the name of a given attribute that the composer class should have.

In [20]:
class Star:
    
    def __init__(self):
        self.whoami = 'I am a star!'
        
class Galaxy:
    
    def __init__(self):
        self.whoami = 'I am a Galaxy!'
    
class myClass:
    
    def __init__(self, composer):
        self.whoami = composer().whoami

inst1 = myClass(Star)
inst2 = myClass(Galaxy)
print("whoami =", inst1.whoami)
print("whoami =", inst2.whoami)

whoami = I am a star!
whoami = I am a Galaxy!


> Note: This will break if the composer does not have an attribute called `whoami`.

Composition also alows preinitialised class instances to be passed.

In [21]:
class Star:
    
    def __init__(self, mag):
        self.mag = mag
        
class Galaxy:
    
    def __init__(self, mag):
        self.mag = mag
    
class myClass:
    
    def __init__(self, composer):
        self.comp = composer
        self.mag = self.comp.mag

        
star = Star(11.05)
inst1 = myClass(star)
inst2 = myClass(Galaxy(8.2))
print("mag =", inst1.mag)
print("mag =", inst2.mag)

mag = 11.05
mag = 8.2


This can be useful, especially if we want to be able to change the original instance attributes.

In [22]:
inst1.mag = 12.3
inst1.comp.mag = 13.1
print("inst1 mag =", inst1.mag)
print("star mag =", star.mag)

inst1 mag = 12.3
star mag = 13.1


## 3 Abstract Classes
---

<br>

Abstract classes are classes that contain abstract methods.

<br>

### Abstract Methods

An abstract method is a method that is defined but not implemented. This is useful for definining parent classes that impose some conditions on the child.

In [23]:
from abc import ABC, abstractmethod

class Parent(ABC):
    
    @abstractmethod
    def get_id(self):
        pass

print ('Parent.__dict__', Parent.__dict__)

Parent.__dict__ {'__module__': '__main__', 'get_id': <function Parent.get_id at 0x1117232f0>, '__dict__': <attribute '__dict__' of 'Parent' objects>, '__weakref__': <attribute '__weakref__' of 'Parent' objects>, '__doc__': None, '__abstractmethods__': frozenset({'get_id'}), '_abc_impl': <_abc_data object at 0x111681b10>}


We can see that the method `get_id` has been defined but does nothing.

We now define a child class that inherits this parent.

In [24]:
class Child(Parent):
    
    def __init__(self):
        self.whoami = 'I am a child!'
        
inst = Child()
print(inst.whoami)

TypeError: Can't instantiate abstract class Child with abstract methods get_id

This raises an error when trying to create an instance. This is because the child class needs to override the abstract method `get_id`. 

In [26]:
class Child(Parent):
    
    def __init__(self):
        self.get_id()
        
    def get_id(self):
        self.whoami = 'I am a child!'
        
inst = Child()
print(inst.whoami)

I am a child!


This allows the parent class, which may be written with no knowledge of the properties of the child class, to impose some conditions on what a given child class should do.

<br>

### Abstract Properties

In the previous notebook we learned how class properties can be defined to impose conditions on the values of certain class attributes. Using Abstract methods we can combine these tools to ensure that a given class has the properties we want.

We start by defining an abstract class with the abstract property `whoami` and its corresponding setter.

In [28]:
class AstroObject(ABC):
    
    @property
    @abstractmethod
    def whoami(self):
        pass
    
    @whoami.setter
    @abstractmethod
    def whoami(self, value):
        pass

We can then define a child class that inherits this abstract class.

In [29]:
class Star(AstroObject):
    
    def __init__(self):
        self.whoami = 'I am a star!'

star = Star()
print("whoami =", star.whoami)

TypeError: Can't instantiate abstract class Star with abstract methods whoami

We can see that while a value for the attribute is provided the property is not explicitly defined. This way we can impose that the `Star` class must define a getter and setter for the attribute `whoami`.

In [30]:
class Star(AstroObject):
    
    def __init__(self):
        self.whoami = 'I am a star!'
    
    @property
    def whoami(self):
        return self._whoami
    
    @whoami.setter
    def whoami(self, value):
        if not isinstance(value, str):
            raise ValueError('whoami must be a string')
        self._whoami = value

star = Star()
print("whoami =", star.whoami)


whoami = I am a star!


This in turn could be used in conjunction with what we learned about class composition to ensure that composers have the properties needed to work with the child class.